In [ ]:
!pip install rioxarray
!pip install geopandas

     |████████████████████████████████| 47 kB 1.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.3 MB 9.0 MB/s 
     |████████████████████████████████| 19.3 MB 1.2 MB/s 
  Created wheel for rioxarray: filename=rioxarray-0.9.1-py3-none-any.whl size=54611 sha256=8d965942ca411995e1d4edc209997f90941322f98bf526f3a01a5630681b351e
  Stored in directory: /root/.cache/pip/wheels/07/da/9e/1cc57b2e7a29a206893db83e984a341e2e94378263e0798229
Successfully built rioxarray
     |████████████████████████████████| 1.0 MB 5.2 MB/s 
     |████████████████████████████████| 16.7 MB 39.1 MB/s 


In [ ]:
import pickle
import os
import numpy as np
import rioxarray
import cv2
import geopandas as gpd
import pandas as pd
from pyproj import CRS
from shapely.geometry import Polygon
from tqdm import tqdm
import time
from random import shuffle
from google.colab import drive
import matplotlib.pyplot as plt
from PIL import Image


drive.mount("/content/drive")
os.chdir('/content/drive/MyDrive/ASU - Zhiang/Projects/instance segmentation/')
os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['instance_registration_IoU_all_adjacents.ipynb',
 'data',
 'instance_registration.ipynb',
 'TGRS',
 'instance_registration_IoU.ipynb',
 'random_ellipse_shapefile_generation.ipynb',
 'tile_splitting.ipynb']

In [ ]:
class Instance_Registration(object):
    def __init__(self, instance_dir, 
                 save_shapefile,
                 tif_height_pixel = 1000,
                 tif_width_pixel = 1000,
                 tif_height_res = -0.01,
                 tif_width_res = 0.01,
                 tile_overlap_ratio=0.1, 
                 detection_threshold=0.75, 
                 segmentation_threshold=0.5, 
                 iou_threshold=0.5, 
                 disable_merge=False,
                 test=True):
        
        assert os.path.exists(instance_dir)
        self.tile_files = [os.path.join(instance_dir, f) for f in os.listdir(instance_dir) if f.endswith('.pickle')]
        self.tile_files.sort()  ########################################################################
        self.tiles = {}
        self.instances = []  # 
        tile_data = self._get_instance(self.tile_files[0])
        self.test = test
        if self.test == True:
            _, self.h, self.w = tile_data['masks'].shape
        else:
            _, _, self.h, self.w = tile_data['masks'].shape
        tif_name = tile_data['image_name']
        tif = rioxarray.open_rasterio(tif_name)
        epsg = tif.rio.crs.to_epsg()
        self.crs = CRS(epsg)
        #_, self.tiff_h, self.tiff_w = tif.shape
        #self.tif_h_size, self.tif_v_size = tif.rio.resolution()
        
        self.tiff_h, self.tiff_w = tif_height_pixel, tif_width_pixel
        self.tif_h_size, self.tif_v_size = tif_width_res, tif_height_res

        self.mask_h_size = self.tif_h_size * self.tiff_h / self.h
        self.mask_v_size = self.tif_v_size * self.tiff_w / self.w
        self.overlap = int(self.h * tile_overlap_ratio)
        self.iou_threshold = iou_threshold
        dir_path = os.path.dirname(os.path.realpath(tif_name))
        tif_name = os.path.join(dir_path, '0_0.tif')
        #assert os.path.isfile(tif_name)
        #tif = rioxarray.open_rasterio(tif_name)
        #self.h_start, self.v_start, _, _= tif.rio.bounds()
        self.h_start, self.v_start = 0, 0

        self.save_shapefile = save_shapefile
        self.detection_threshold = detection_threshold
        self.segmentation_threshold = segmentation_threshold
        self.disable_merge = disable_merge
        self.twins = []
        self.instance_num = 0
        
        
    def start_registration(self):
        updated_tile_files = []
        timestamps = []
        self.instances = []
        self.tiles = {}
        print("Instance registration: ")
        start_time = time.time()
        for tile_file in tqdm(self.tile_files):
            tile_data = self._get_instance(tile_file)
            masks = tile_data['masks']
            instance_N = masks.shape[0]
            if instance_N == 0:
                continue
            updated_tile_files.append(tile_file)
            if not self.test:
                masks = np.squeeze(masks, axis=1)
            tif_name = tile_data['image_name']
            tile_indices = tuple([int(i) for i in tif_name.split('/')[-1].split('.')[0].split('_')])
            # post processing: detection confidence filter
            detect_scores = tile_data['scores']
            id_strs = tile_data['ids']  ########################################################
            # prune data by detection_threshold
            masks = masks[detect_scores>self.detection_threshold]
            detect_scores = detect_scores[detect_scores>self.detection_threshold]
            id_strs = id_strs[detect_scores>self.detection_threshold]  ######################################################## 
            tif = rioxarray.open_rasterio(tif_name) 
            for idx, mask in enumerate(masks):
                # post processing: segmentation confidence filter
                mask = mask > self.segmentation_threshold
                # post processing: contour analysis
                contours, _ = cv2.findContours(mask.astype(np.uint8).copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
                if len(contours) > 1:
                    areas = [cv2.contourArea(cnt) for cnt in contours]
                    i = np.argmax(areas)
                    contour = contours[i]
                    mask = np.zeros_like(mask).astype(np.uint8)
                    cv2.fillPoly(mask, pts =[contour], color=(255))
                    local_mask = mask > 0
                else:
                    contour = contours[0]
                    local_mask = mask
                # get location using pixel coords: top, bottom, left, right, middle, top-left, top-right, bottom-left, bottom-right
                contour = np.squeeze(contour, axis=1)
                locations = self._get_locations(contour, local_mask, tile_indices)
                # convert masks to global mask
                global_mask = self._convert_global_mask(local_mask, tile_indices)
                # get global bbox
                global_bbox = self._get_global_bbox(global_mask)
                # instance registration
                instance = {'locations': locations, 
                            'score': detect_scores[idx], 
                            'global_bbox': global_bbox, 
                            'global_mask': global_mask,
                            'id_str': id_strs[idx],}

                if self.disable_merge:
                    self.instances.append(instance)
                else:
                    self._instance_registration(instance)
            timestamps.append(time.time() - start_time)

        self.instance_num = len(self.instances)
        self.clean_twin_instances()   
        print("Instance #: ", self.instance_num)
        return updated_tile_files, timestamps
    
    
    def update_tile_files(self, updated_tile_files):
        self.tile_files = updated_tile_files
    
                
    def save(self):
        print('save shapefile: ')
        dataframesList = []
        for instance in tqdm(self.instances):
            if instance is None:
                continue
            geodataframe = gpd.GeoDataFrame(pd.DataFrame({'score': [instance['score']], 'id': instance['id_str']}), 
                                           crs=self.crs, 
                                           geometry=[self._convert_mask_to_poly(instance)])
            dataframesList.append(geodataframe)
        rdf = gpd.GeoDataFrame(pd.concat(dataframesList, ignore_index=True))
        rdf.to_file(self.save_shapefile)
                
    def _instance_registration(self, instance):

        (x,y) = list(instance['locations'].keys())[0]
        if 'middle' in instance['locations'][(x, y)].keys():
            self._add_instance(instance)
            return None
        
        merged = []
        for location in instance['locations'][(x, y)]:
            if location == 'left':
                adjacent_indices = (x-1, y)
                adjacent_location = 'right'
                merged.append(self._merge_instance(instance, location, adjacent_indices, adjacent_location))
            if location == 'right':
                adjacent_indices = (x+1, y)
                adjacent_location = 'left'
                merged.append(self._merge_instance(instance, location, adjacent_indices, adjacent_location))
            if location == 'top':
                adjacent_indices = (x, y+1)
                adjacent_location = 'bottom'
                merged.append(self._merge_instance(instance, location, adjacent_indices, adjacent_location))
            if location == 'bottom':
                adjacent_indices = (x, y-1)
                adjacent_location = 'top'
                merged.append(self._merge_instance(instance, location, adjacent_indices, adjacent_location))
            if location == 'top-left':
                adjacent_indices = (x-1, y+1)
                adjacent_location = 'bottom-right'
                merged.append(self._merge_instance(instance, location, adjacent_indices, adjacent_location))
            if location == 'top-right':
                adjacent_indices = (x+1, y+1)
                adjacent_location = 'bottom-left'
                merged.append(self._merge_instance(instance, location, adjacent_indices, adjacent_location))
            if location == 'bottom-left':
                adjacent_indices = (x-1, y-1)
                adjacent_location = 'top-right'
                merged.append(self._merge_instance(instance, location, adjacent_indices, adjacent_location))
            if location == 'bottom-right':
                adjacent_indices = (x+1, y-1)
                adjacent_location = 'top-left'
                merged.append(self._merge_instance(instance, location, adjacent_indices, adjacent_location))
                
        merged = np.array(merged)
        merged_num = np.unique(merged[merged>=0]).shape[0]
        if merged_num == 0:
            self._add_instance(instance)
        elif merged_num > 1:
            self.twins.append(np.unique(merged[merged>=0]))
            
                
    def _add_instance(self, instance):
        self.instances.append(instance)
        instance_id = len(self.instances) - 1
        # update tile table
        tile_indices = list(instance['locations'].keys())[0]
        if not self.tiles.get(tile_indices, False):
            # initialize tile
            empty_locations = {'left':[], 'right':[], 'top':[], 'bottom':[], 'middle':[], 'top-left':[], 'top-right':[], 'bottom-left':[], 'bottom-right':[]}
            self.tiles[tile_indices] = empty_locations
        # update tile
        tile_indices = list(instance['locations'].keys())[0]
        for location in instance['locations'][tile_indices].keys():
            self.tiles[tile_indices][location].append(instance_id)
                        
    def _merge_instance(self, instance, location, adjacent_indices, adjacent_location):
        if not self.tiles.get(adjacent_indices, False): 
            return -1
        else:
            for adjacent_id in self.tiles[adjacent_indices][adjacent_location]:
                adjacent_instance = self.instances[adjacent_id]
                if self._bbox_intersection(instance['global_bbox'], adjacent_instance['global_bbox']):
                    tile_indices = list(instance['locations'].keys())[0]
                    mask1 = instance['locations'][tile_indices][location]
                    mask2 = adjacent_instance['locations'][adjacent_indices].get(adjacent_location, None)
                    iou = self._mask_IoU(mask1, mask2)
                    if iou > self.iou_threshold:
                        merged_mask = np.concatenate((instance['global_mask'], adjacent_instance['global_mask']))
                        merged_mask = np.unique(merged_mask, axis=0)
                        merged_mask = tuple(map(tuple, merged_mask))

                        self.instances[adjacent_id]['global_mask'] = merged_mask
                        self.instances[adjacent_id]['global_bbox'] = self._get_global_bbox(merged_mask)
                        self.instances[adjacent_id]['score'] = np.max((self.instances[adjacent_id]['score'], instance['score']))
                        self.instances[adjacent_id]['id_str'] += ','+ instance['id_str'] ########################################################
                        merged_mask_partial = np.concatenate((mask1, mask2))
                        merged_mask_partial = np.unique(merged_mask_partial, axis=0)
                        merged_mask_partial = tuple(map(tuple, merged_mask_partial))

                        # after merging, instance belongs to multiple tiles. instance's new tile should be updated
                        if self.instances[adjacent_id]['locations'].get(tile_indices, False):
                            if self.instances[adjacent_id]['locations'][tile_indices].get(location, False):
                                existing_mask = self.instances[adjacent_id]['locations'][tile_indices][location]
                                temp_mask = np.concatenate((existing_mask, merged_mask_partial))
                                temp_mask = np.unique(temp_mask, axis=0)
                                temp_mask = tuple(map(tuple, temp_mask))
                                self.instances[adjacent_id]['locations'][tile_indices][location] = temp_mask
                            else:
                                self.instances[adjacent_id]['locations'][tile_indices] = {location: merged_mask_partial}
                        else:
                            self.instances[adjacent_id]['locations'][tile_indices] = {}
                            for loc, mask_partial in instance['locations'][tile_indices].items():
                                if loc == location:
                                    self.instances[adjacent_id]['locations'][tile_indices][location] = merged_mask_partial
                                else:
                                    self.instances[adjacent_id]['locations'][tile_indices][loc] = mask_partial

                            
                        
                        # tile is also expanded as instance merge
                        if not self.tiles.get(tile_indices, False):
                            empty_locations = {'left':[], 'right':[], 'top':[], 'bottom':[], 'middle':[], 'top-left':[], 'top-right':[], 'bottom-left':[], 'bottom-right':[]}
                            self.tiles[tile_indices] = empty_locations
                        for location in instance['locations'][tile_indices].keys():
                            self.tiles[tile_indices][location].append(adjacent_id)
                        return adjacent_id   
        return -1

    def clean_twin_instances(self):
        for twins in self.twins:
            twins.sort()
            for i in twins[1:]:
                self.merge_twins(twins[0], i)
    
    def merge_twins(self, id1, id2):
        instance1 = self.instances[id1]
        instance2 = self.instances[id2]
        mask1 = instance1['global_mask']
        mask2 = instance2['global_mask']
        merged_mask = np.concatenate((mask1, mask2))
        merged_mask = np.unique(merged_mask, axis=0)
        merged_mask = tuple(map(tuple, merged_mask))
        self.instances[id1]['global_mask'] = merged_mask
        self.instances[id1]['global_bbox'] = self._get_global_bbox(merged_mask)
        self.instances[id2] = None
        self.instance_num -= 1

    def _mask_IoU(self, mask1, mask2):
        if mask2 == None:
            return -1
        mask_hash1 = [hash(mask_indices) for mask_indices in mask1]
        mask_hash2 = [hash(mask_indices) for mask_indices in mask2]
        intersection = np.count_nonzero(np.in1d(mask_hash1, mask_hash2, assume_unique=True))
        union = np.unique(mask_hash1 + mask_hash2).shape[0]
        return intersection / union

    def _mask_intersection_hash_method(self, mask1, mask2):
        mask_hash1 = [hash(mask_indices) for mask_indices in mask1]
        mask_hash2 = [hash(mask_indices) for mask_indices in mask2]
        return -np.count_nonzero(np.in1d(mask_hash1, mask_hash2, assume_unique=True)) * self.mask_h_size * self.mask_v_size
    
    def _mask_intersection_brute_force_method(self, mask1, mask2):
        overlap = len([px for px in mask1 if px in mask2])
        return -overlap * self.mask_h_size * self.mask_v_size
    
        
    def _convert_global_mask(self, mask, tile_indices):
        x = tile_indices[0] * (self.w - self.overlap)
        y = tile_indices[1] * (self.h - self.overlap) + self.h
        return tuple([(x+i, y-j) for j,i in np.asarray(np.nonzero(mask)).transpose()])
    
    def _get_global_bbox(self, mask):
        xmin, ymin = np.asarray(mask).min(axis=0)
        xmax, ymax = np.asarray(mask).max(axis=0)
        return [xmin, ymin, xmax, ymax]
    
    def _bbox_intersection(self, bbox1, bbox2):
        (xmin_a, ymin_a, xmax_a, ymax_a) = bbox1
        (xmin_b, ymin_b, xmax_b, ymax_b) = bbox2
        if xmin_a < xmax_b <= xmax_a and (ymin_a < ymax_b <= ymax_a or ymin_a <= ymin_b < ymax_a):
            return True
        elif xmin_a <= xmin_b < xmax_a and (ymin_a < ymax_b <= ymax_a or ymin_a <= ymin_b < ymax_a):
            return True
        elif xmin_b < xmax_a <= xmax_b and (ymin_b < ymax_a <= ymax_b or ymin_b <= ymin_a < ymax_b):
            return True
        elif xmin_b <= xmin_a < xmax_b and (ymin_b < ymax_a <= ymax_b or ymin_b <= ymin_a < ymax_b):
            return True
        else:
            return False
        
    def _get_instance(self, tile_file):
        with open(tile_file, 'rb') as handle:
            tile_data = pickle.load(handle)
        return tile_data
    
    def _get_locations(self, contour, local_mask, tile_indices):
        x1, y1 = np.min(contour, axis=0)
        x2, y2 = np.max(contour, axis=0)
        locations = {}
        if x1 < self.overlap:
            mask = local_mask.copy()
            mask[:, self.overlap:] = 0
            mask_global = self._convert_global_mask(mask, tile_indices)
            locations['left'] = mask_global
        if x2 > self.h - self.overlap:
            mask = local_mask.copy()
            mask[:, :-self.overlap] = 0
            mask_global = self._convert_global_mask(mask, tile_indices)
            locations['right'] = mask_global
        if y1 < self.overlap:
            mask = local_mask.copy()
            mask[self.overlap:,:] = 0
            mask_global = self._convert_global_mask(mask, tile_indices)
            locations['top'] = mask_global
        if y2 > self.w - self.overlap:
            mask = local_mask.copy()
            mask[:-self.overlap, :] = 0
            mask_global = self._convert_global_mask(mask, tile_indices)
            locations['bottom'] = mask_global
        if len(locations) == 0:
            mask = local_mask.copy()
            mask_global = self._convert_global_mask(mask, tile_indices)
            locations['middle'] = mask_global
        
        location_list = locations.keys()
        if ('left' in location_list) & ('top' in location_list):
            mask = local_mask.copy()
            mask[:, self.overlap:] = 0
            mask[self.overlap:,:] = 0
            mask_global = self._convert_global_mask(mask, tile_indices)
            locations['top-left'] = mask_global

        if ('left' in location_list) & ('bottom' in location_list):
            mask = local_mask.copy()
            mask[:, self.overlap:] = 0
            mask[:-self.overlap, :] = 0
            mask_global = self._convert_global_mask(mask, tile_indices)
            locations['bottom-left'] = mask_global
            
        if ('right' in location_list) & ('top' in location_list):
            mask = local_mask.copy()
            mask[:, :-self.overlap] = 0
            mask[self.overlap:,:] = 0
            mask_global = self._convert_global_mask(mask, tile_indices)
            locations['top-right'] = mask_global

        if ('right' in location_list) & ('bottom' in location_list):
            mask = local_mask.copy()
            mask[:, :-self.overlap] = 0
            mask[:-self.overlap, :] = 0
            mask_global = self._convert_global_mask(mask, tile_indices)
            locations['bottom-right'] = mask_global

        return {tile_indices: locations}
        
        
    def _convert_mask_to_poly(self, instance):
        mask = np.asarray(instance['global_mask'])
        bbox = instance['global_bbox']
        bottom_left = np.asarray(mask).min(axis=0)
        local_mask = mask - bottom_left
        #print(local_mask.min(axis=0))
        #print(local_mask.max(axis=0))
        mask_shape = (int(bbox[2] - bbox[0]) + 1, int(bbox[3] - bbox[1]) + 1)  
        local_mask = self._create_bool_mask(local_mask, mask_shape)
        contours, _ = cv2.findContours(local_mask.astype(np.uint8).copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        assert len(contours) == 1
        contour = np.asarray(contours[0]) + bottom_left
        contour = contour.reshape(-1, 2).tolist()
        coords = [(self.h_start + pixel[0]*self.mask_h_size, 
                   self.v_start - pixel[1]*self.mask_v_size) 
                  for pixel in contour]
        coords = np.asarray(coords)
        poly = Polygon(zip(coords[:, 0].tolist(), coords[:, 1].tolist()))
        return poly

        
    def _create_bool_mask(self, mask, size):
        """
        :param mask: mask by index
        :param size: size of image
        :return: bool mask
        """
        mask_ = np.zeros(size)
        mask = mask.tolist()
        for x, y in mask:
            #if (x < size[0]) & (y < size[1]):
            mask_[int(x), int(y)] = 1
        return mask_.transpose() 

In [ ]:
ir = Instance_Registration('data/random_generation/split/', 
                           'data/random_generation/merged_400_3.shp', 
                           iou_threshold=0.88, 
                           disable_merge=False)

In [ ]:
updated_tile_files, timestamps = ir.start_registration()
ir.save()

Instance registration: 


100%|██████████| 16/16 [04:53<00:00, 18.34s/it]


Instance #:  3600
save shapefile: 


100%|██████████| 3605/3605 [00:49<00:00, 72.15it/s]


In [ ]:
ir.twins

[array([1823, 2546]),
 array([1951, 2707]),
 array([2859, 3453]),
 array([2931, 3459]),
 array([3187, 3512])]

In [ ]:
## if save failed
"""
test = Instance_Registration('data/random_generation/split/', 
                           'data/random_generation/merged_15_5.shp', 
                           disable_merge=False)

test.instances = ir.instances[1:]
test.save()
"""

# evaluation

$Accuracy = \frac{TP+TN}{TP+TN+FP+TN}$  
$Precision = \frac{TP}{TP+FP}$  
$Recall = \frac{TP}{TP+FN}$  

Positive indicates merged and negative indicates unmerged in the resulted shapefile. 

$TP$: (1) $merged_i = merged_j$ for all {i, j} in merged ids, **and** (2) $IoU(ground\_truth, merged) > C$ 

$TN$: $IoU(ground\_truth, unmerged) > C$

$FP$: (1) $merged_i \neq merged_j$ for any {i, j} in merged ids, **or** (2) $IoU(ground\_truth, merged) \leq C$

$FN$: $IoU(ground\_truth, unmerged)\leq C$

In [ ]:
def IoU(poly1, poly2):
    i = poly1.intersection(poly2).area
    u = poly1.union(poly2).area
    return i/u

def uniqueness(TP, TN):
    a = TP + TN
    unq, unq_idx, unq_cnt = np.unique(a, return_inverse=True, return_counts=True)
    cnt_mask = unq_cnt > 1
    dup_ids = unq[cnt_mask]
    cnt_idx, = np.nonzero(cnt_mask)
    idx_mask = np.in1d(unq_idx, cnt_idx)
    idx_idx, = np.nonzero(idx_mask)
    srt_idx = np.argsort(unq_idx[idx_mask])
    dup_idx = np.split(idx_idx[srt_idx], np.cumsum(unq_cnt[cnt_mask])[:-1])
    duplicated_ids = [a[i[0]] for i in dup_idx if i.size !=0]
    return duplicated_ids

def evaluate(groundtruth_shapefile, merged_shapefile, IoU_threshold=0.88):
    assert os.path.exists(groundtruth_shapefile)
    assert os.path.exists(merged_shapefile)
    gt_shp = gpd.read_file(groundtruth_shapefile)
    merged_shp = gpd.read_file(merged_shapefile)
    N, _ = merged_shp.shape
    TP = []
    FP = []
    TN = []
    FN = []
    for i in range(N):
        merged_poly = merged_shp['geometry'][i]
        ids = merged_shp['id'][i]
        if ',' in ids:
            # merged result: postive
            ids = [int(id) for id in ids.split(',')]
            if np.unique(ids).shape[0] == 1:
                id = ids[0]
                gt_poly = gt_shp['geometry'][id]
                if IoU(gt_poly, merged_poly) > IoU_threshold:
                    # true positive
                    TP.append(id)
                else:
                    # false positive
                    FP.append(id)
            else:
                # false positive
                FP.append(ids)
        else:
            # unmerged result: negative
            id = int(ids)
            gt_poly = gt_shp['geometry'][id]
            if IoU(gt_poly, merged_poly) > IoU_threshold:
                # true negative
                TN.append(id)
            else:
                # false negative
                FN.append(id)
    


    tp = len(TP)
    tn = len(TN)
    fp = len(FP)
    fn = len(FN)
    print("total: ", N)
    print("TP, TN, FP, FN: ", tp, tn, fp, fn)
    print("FP: ", FP)
    print("FN: ", FN)
    print("Duplicated ids: ", uniqueness(TP, TN))
    print("accuracy: {acc:.2f}%".format(acc=(tp+tn)/(tp+tn+fp+fn)*100))
    print("precision: {pre:.2f}%".format(pre=tp/(tp+fp)*100))
    print("recall: {rec:.2f}%".format(rec=tp/(tp+fn)*100))



In [ ]:
evaluate('data/random_generation/0_0.shp', 'data/random_generation/merged_400_3.shp')

total:  3600
TP, TN, FP, FN:  1208 2391 1 0
FP:  [[63, 63, 1788]]
FN:  []
Duplicated ids:  []
accuracy: 99.97%
precision: 99.92%
recall: 100.00%


In [ ]:
ir.twins

[array([1219, 1960])]

# Intersection comparison

https://gis.stackexchange.com/questions/411777/how-is-polygon-intersection-implemented-in-jts-shapely

https://en.wikipedia.org/wiki/Weiler%E2%80%93Atherton_clipping_algorithm

https://stackoverflow.com/questions/2272179/a-simple-algorithm-for-polygon-intersection#:~:text=Compute%20the%20center%20of%20mass,the%20two%20polygons%20%22intersect%22. 

In [ ]:
!rm data/random_generation/split/*.pickle

In [ ]:
merged = [-1, 6, 3, 5, -1]
merged = np.array(merged)
a = merged[merged>=0]
a.sort()
for i in a[1:]:
    print(i)

5
6


3